# De-Duping Data

## Instructions

In this exercise, we're doing ETL on a file we've received from some customer. That file contains data about people, including:

* first, middle and last names
* gender
* birth date
* Social Security number
* salary

But, as is unfortunately common in data we get from this customer, the file contains some duplicate records. Worse:

* In some of the records, the names are mixed case (e.g., "Carol"), while in others, they are uppercase (e.g., "CAROL"). 
* The Social Security numbers aren't consistent, either. Some of them are hyphenated (e.g., "992-83-4829"), while others are missing hyphens ("992834829").

The name fields are guaranteed to match, if you disregard character case, and the birth dates will also match. (The salaries will match, as well,
and the Social Security Numbers *would* match, if they were somehow put in the same format).

Your job is to remove the duplicate records. The specific requirements of your job are:

* Remove duplicates. It doesn't matter which record you keep; it only matters that you keep one of them.
* Preserve the data format of the columns. For example, if you write the first name column in all lower-case, you haven't met this requirement.
* Write the result as a Parquet file, as designated by *destFile*.
* The final Parquet "file" must contain multiple part files (ending in ".parquet").

**Hint:** <br/>
The initial dataset contains 103,000 records.<br/>
The de-duplicated result haves 100,000 records.

## Getting Started

In [0]:
from pyspark.sql import SparkSession

In [0]:
# Initialize Spark Session
spark = (SparkSession.builder
         .appName("Deduplicate Data")
         .getOrCreate())

**Data Source**

In [0]:
%run ../DatasetSourcePath

In [0]:
sourceFile = sourcePath + "/dataset/people-with-dups.txt"
destFile = "dbfs:/FileStore/out/people.parquet"

**Set the ```shuffle.partitions```**

In [0]:
# dropDuplicates() will likely introduce a shuffle, so it helps to reduce the number of post-shuffle partitions.
spark.conf.set("spark.sql.shuffle.partitions", 7)

**Read the data**

In [0]:
# Okay, now we can read this thing.

df = (spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("sep", ":")
    .csv(sourceFile)
)

In [0]:
df.sample(True, 0.1).show(20)

**Drop the duplicate record**

In [0]:
from pyspark.sql.functions import *

In [0]:
(df
  .select(col("*"),
      lower(col("firstName")).alias("lcFirstName"),
      lower(col("lastName")).alias("lcLastName"),
      lower(col("middleName")).alias("lcMiddleName"),
      translate(col("ssn"), "-", "").alias("ssnNums")
   ).show())

In [0]:
from pyspark.sql.functions import *

dedupedDF = (df
  .select(col("*"),
      lower(col("firstName")).alias("lcFirstName"),
      lower(col("lastName")).alias("lcLastName"),
      lower(col("middleName")).alias("lcMiddleName"),
      translate(col("ssn"), "-", "").alias("ssnNums")
   )
  .dropDuplicates(["lcFirstName", "lcMiddleName", "lcLastName", "ssnNums", "gender", "birthDate", "salary"])
  .drop("lcFirstName", "lcMiddleName", "lcLastName", "ssnNums")
)

In [0]:
dedupedDF.show(15)

In [0]:
df.count()

In [0]:
dedupedDF.count()

**Save the data as parquest**

In [0]:
# Now we can save the results. We'll also re-read them and count them, just as a final check.
(dedupedDF.write
   .mode("overwrite")
   .parquet(destFile)
)

In [0]:
file_list = dbutils.fs.ls(destFile)
for f in file_list:
  print(f.name)

In [0]:
# import os

# # List files in the directory
# file_list = os.listdir(destFile)

# # Display the list of files
# print("\n".join(file_list))

In [0]:
# spark.stop()